# HCR Post-alignment Image Processing

In [1]:
import os
from skimage import exposure
from skimage.filters.rank import mean_bilateral
from skimage.restoration import  denoise_bilateral
from skimage.morphology import disk
from skimage.filters import gaussian
import cv2 as cv
import numpy as np
import tifffile as tiff
import SimpleITK as sitk
from scipy import ndimage

In [2]:
file_path = r'C:\Users\keshavgubbi\Desktop\HCR\raw_data\aligned\ccka_4'
#C:\Users\keshavgubbi\Desktop\HCR\raw_data\aligned\chat

In [3]:
def split_and_rename(f):
    filename, exte = f.split('.')
    return filename

#Contrast Enhancement Function
alpha = 10.0 # Contrast control (1.0-3.0) but 5 is required for my purposes here
beta = 5 # Brightness control (0-100). Not to be added beyond 5, to not hamper the signal with unnecesary salt and peper noise.

In [4]:

for file in os.listdir(file_path):
    if file.endswith('.nrrd'):
        print(file)
        processed_page_list = []
        name = split_and_rename(file)
        aligned_image = sitk.ReadImage(os.path.join(file_path, file))
        aligned_image_array_list = list(sitk.GetArrayFromImage(aligned_image))
        
        # we now have a list of 2D images and I can do processing on them and then restack them.
        print(len(aligned_image_array_list))
        for image in aligned_image_array_list:
            # print(image.shape)
            contrast_enhanced_page = cv.convertScaleAbs(image, alpha=alpha, beta=beta) # contrst enhancement, doen for every 2D Image in the stack            
#             bilat_page= cv.bilateralFilter(contrast_enhanced_page,9,75,75) #Adding a Bilat filter to reduce salt and pepper noise
#             min_filter_image = ndimage.minimum_filter(bilat_page, size=1) # passing through a mimimum filter 
            processed_page_list.append(contrast_enhanced_page)
            
        # print(len(processed_page_list))
        processed_image_stack = np.stack(processed_page_list)
        processed_image = np.stack(processed_image_stack).astype('uint8')

        print(f'Writing image {name}.tif')
        with tiff.TiffWriter(os.path.join(file_path, f"{name}_processed.tif"), imagej=True) as tifw:
            tifw.write(processed_image, metadata={'spacing': 1.0, 'unit': 'um', 'axes': 'ZYX'})

T_ccka.nrrd
359
Writing image T_ccka.tif
T_ccka_4_gcamp.nrrd
359
Writing image T_ccka_4_gcamp.tif
